In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

In [ ]:
import numpy as np
import torch
import time
from quantem.widget import Show3D

def generate_focal_series(n_frames, size, device="cpu"):
    t0 = time.time()
    fy = torch.fft.fftfreq(size, device=device).unsqueeze(1)
    fx = torch.fft.fftfreq(size, device=device).unsqueeze(0)
    freq_r2 = fy ** 2 + fx ** 2
    spacing = size / 10
    atom_sigma = spacing * 0.06
    impulses = torch.zeros(size, size, device=device)
    for i in range(-2, 15):
        for j in range(-2, 15):
            cx = i * spacing + (j % 2) * spacing * 0.5
            cy = j * spacing * 0.866
            ix, iy = int(round(cx)), int(round(cy))
            if 0 <= ix < size and 0 <= iy < size:
                impulses[iy, ix] = 1.0
    gauss_atom = torch.exp(-2 * np.pi ** 2 * atom_sigma ** 2 * freq_r2)
    lattice_fft = torch.fft.fft2(impulses) * gauss_atom
    y = torch.linspace(0, size - 1, size, device=device)
    x = torch.linspace(0, size - 1, size, device=device)
    yy, xx = torch.meshgrid(y, x, indexing="ij")
    r_center = torch.sqrt((xx - size / 2) ** 2 + (yy - size / 2) ** 2)
    ring = torch.exp(-((r_center - size * 0.35) / (size * 0.01)) ** 2) * 0.3
    lattice_fft = lattice_fft + torch.fft.fft2(ring)
    N_KEY = 50
    key_abs = torch.linspace(0, 1, N_KEY, device=device)
    key_sigmas = (key_abs ** 2 * spacing * 0.5).view(N_KEY, 1, 1)
    key_filters = torch.exp(-2 * np.pi ** 2 * key_sigmas ** 2 * freq_r2.unsqueeze(0))
    key_frames = torch.fft.ifft2(lattice_fft.unsqueeze(0) * key_filters).real
    r_norm_sq = (r_center / size) ** 2
    defocus = torch.linspace(-1, 1, n_frames, device=device)
    abs_defocus = torch.abs(defocus)
    idx_float = abs_defocus * (N_KEY - 1)
    idx_low = idx_float.long().clamp(max=N_KEY - 2)
    frac = idx_float - idx_low.float()
    result = torch.empty(n_frames, size, size)
    batch_size = min(200, n_frames)
    for b0 in range(0, n_frames, batch_size):
        b1 = min(b0 + batch_size, n_frames)
        nb = b1 - b0
        df = defocus[b0:b1]
        bf = frac[b0:b1].view(nb, 1, 1)
        bi = idx_low[b0:b1]
        batch = torch.lerp(key_frames[bi], key_frames[bi + 1], bf)
        fringe_amp = torch.abs(df).view(nb, 1, 1) * 0.3
        fringe_k = (1.0 + torch.abs(df) * 30.0).view(nb, 1, 1)
        result[b0:b1] = batch + fringe_amp * torch.cos(fringe_k * r_norm_sq.unsqueeze(0) * 2 * np.pi) + 1.0
    elapsed = time.time() - t0
    print(f"Generated {n_frames}x{size}x{size} in {elapsed:.1f}s ({result.nelement() * 4 / 1e6:.0f} MB)")
    return result

## Test 1: 1000 x 250x250 (250 MB)
- Frame: 0.25 MB | Buffer: 64 frames (16 MB) | Target: 60 FPS

In [ ]:
data = generate_focal_series(1000, 250)
w = Show3D(data, title="1000 x 250x250", fps=30, pixel_size=0.2)
print(f"Buffer: {w._buffer_size} frames ({w._buffer_size * 250 * 250 * 4 / 1e6:.0f} MB)")
w

## Test 2: 1000 x 500x500 (1 GB)
- Frame: 1 MB | Buffer: 64 frames (64 MB) | Target: 30 FPS

In [ ]:
data = generate_focal_series(1000, 500)
w = Show3D(data, title="1000 x 500x500", fps=30, pixel_size=0.1)
print(f"Buffer: {w._buffer_size} frames ({w._buffer_size * 500 * 500 * 4 / 1e6:.0f} MB)")
w

## Test 3: 100 x 1000x1000 (400 MB)
- Frame: 4 MB | Buffer: 16 frames (64 MB) | Target: 30 FPS

In [ ]:
data = generate_focal_series(100, 1000)
w = Show3D(data, title="100 x 1000x1000", fps=30, pixel_size=0.1)
print(f"Buffer: {w._buffer_size} frames ({w._buffer_size * 1000 * 1000 * 4 / 1e6:.0f} MB)")
w

## Test 4: 50 x 2000x2000 (800 MB)
- Frame: 16 MB | Buffer: 8 frames (128 MB) | Target: 30 FPS

In [ ]:
data = generate_focal_series(50, 2000)
w = Show3D(data, title="50 x 2000x2000", fps=30, pixel_size=0.05)
print(f"Buffer: {w._buffer_size} frames ({w._buffer_size * 2000 * 2000 * 4 / 1e6:.0f} MB)")
w

## Test 5: 10 x 4000x4000 (640 MB)
- Frame: 64 MB | Buffer: 8 frames (512 MB) | Target: 30 FPS

In [ ]:
data = generate_focal_series(10, 4000)
w = Show3D(data, title="10 x 4000x4000", fps=30, pixel_size=0.05)
print(f"Buffer: {w._buffer_size} frames ({w._buffer_size * 4000 * 4000 * 4 / 1e6:.0f} MB)")
w